In [1]:
import numpy as np
import networkx as nx
import pickle
from sklearn.model_selection import train_test_split
import os
#os.chdir(os.curdir + '/work/project/code')
from part import Part
from evaluation import MyPredictionModel, evaluate
from graph import Graph
from typing import List, Set
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [2]:
def graph_to_node_features_and_adjacency_matrix(graph, max_nodes):
    node_features = np.eye(max_nodes)[:len(graph.get_nodes())]
    adj_matrix = nx.to_numpy_array(graph.to_nx())
    padded_features = np.zeros((max_nodes, max_nodes))
    padded_features[:node_features.shape[0], :node_features.shape[1]] = node_features
    padded_matrix = np.zeros((max_nodes, max_nodes))
    padded_matrix[:adj_matrix.shape[0], :adj_matrix.shape[1]] = adj_matrix
    return padded_features, padded_matrix

def prepare_data(graphs, max_nodes):
    node_features_list = []
    adj_matrices_list = []
    for g in graphs:
        node_features, adj_matrix = graph_to_node_features_and_adjacency_matrix(g, max_nodes)
        node_features_list.append(node_features)
        adj_matrices_list.append(adj_matrix)
    return np.array(node_features_list), np.array(adj_matrices_list)

# Load train data
with open('data/graphs.dat', 'rb') as file:
    train_graphs: List[Graph] = pickle.load(file)

max_nodes = max(len(g.get_nodes()) for g in train_graphs)
node_features, adj_matrices = prepare_data(train_graphs, max_nodes)

# Split data
X_train, X_test, y_train, y_test = train_test_split(node_features, adj_matrices, test_size=0.2, random_state=42)

In [3]:
class FFNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = x.view(-1, max_nodes, max_nodes)
        return x

input_size = max_nodes * max_nodes
output_size = max_nodes * max_nodes

print(input_size)
print(output_size)

model = FFNModel(input_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

441
441


In [4]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Evaluation
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f'Test Loss: {test_loss/len(test_loader)}')

Epoch 1/50, Loss: 0.08466651312352616
Epoch 2/50, Loss: 0.07022551001854054
Epoch 3/50, Loss: 0.06925440865058091
Epoch 4/50, Loss: 0.06874116810579453
Epoch 5/50, Loss: 0.06844644632030239
Epoch 6/50, Loss: 0.06816877983510494
Epoch 7/50, Loss: 0.06803414382801581
Epoch 8/50, Loss: 0.06790907364872728
Epoch 9/50, Loss: 0.06785051119730784
Epoch 10/50, Loss: 0.06776467194333405
Epoch 11/50, Loss: 0.0677009848944549
Epoch 12/50, Loss: 0.06764131189260539
Epoch 13/50, Loss: 0.06762342727793161
Epoch 14/50, Loss: 0.06758301408701045
Epoch 15/50, Loss: 0.06754603388128422
Epoch 16/50, Loss: 0.06755023910647331
Epoch 17/50, Loss: 0.06750224817103596
Epoch 18/50, Loss: 0.067477691985373
Epoch 19/50, Loss: 0.06746233012994558
Epoch 20/50, Loss: 0.06743386025310204
Epoch 21/50, Loss: 0.06741073788360669
Epoch 22/50, Loss: 0.06743039254526403
Epoch 23/50, Loss: 0.06760146892145544
Epoch 24/50, Loss: 0.06737163468634569
Epoch 25/50, Loss: 0.0673932199062626
Epoch 26/50, Loss: 0.0674002357227828


In [9]:
class FFNPredictionModel(MyPredictionModel):
    def __init__(self, model, max_nodes):
        self.model = model
        self.max_nodes = max_nodes

    def predict_graph(self, parts: Set[Part]) -> Graph:
        node_features = np.eye(self.max_nodes)[:len(parts)]
        padded_features = np.zeros((self.max_nodes, self.max_nodes))
        padded_features[:node_features.shape[0], :node_features.shape[1]] = node_features
        padded_features_tensor = torch.tensor(padded_features, dtype=torch.float32).unsqueeze(0)
        self.model.eval()
        with torch.no_grad():
            predicted_adj_matrix = self.model(padded_features_tensor).squeeze(0).numpy()
        
        predicted_graph = Graph()
        nodes = list(parts)
        
        # Add edges to the predicted graph
        for i in range(len(nodes)):
            # Find the index of the highest value in the row
            j = np.argmax(predicted_adj_matrix[i, :])
            if i != j and j < len(nodes):  # Ensure we don't add self-loops and j is within bounds
                predicted_graph.add_undirected_edge(nodes[i], nodes[j])
        
        return predicted_graph

# Load the final model
prediction_model = FFNPredictionModel(model, max_nodes)

# For illustration, we compute the eval score on a portion of the training data
instances = [(graph.get_parts(), graph) for graph in train_graphs[:1000]]
eval_score = evaluate(prediction_model, instances)

print(f'Evaluation score: {eval_score:.2f}%')

Evaluation score: 69.83%


In [59]:
from typing import Set
import numpy as np
import torch
from part import Part

class FFNPredictionModel:
    def __init__(self, model, max_nodes, threshold=0.35):
        self.model = model
        self.max_nodes = max_nodes
        self.threshold = threshold

    def predict_adjacency_matrix(self, parts: Set[Part]) -> np.ndarray:
        # Create one-hot encoded node features
        node_features = np.eye(self.max_nodes)[:len(parts)]
        
        # Pad the node features to ensure they have the same size as max_nodes
        padded_features = np.zeros((self.max_nodes, self.max_nodes))
        padded_features[:node_features.shape[0], :node_features.shape[1]] = node_features
        
        # Convert to tensor and add batch dimension
        padded_features_tensor = torch.tensor(padded_features, dtype=torch.float32).unsqueeze(0)
        
        # Set the model to evaluation mode
        self.model.eval()
        
        # Predict the adjacency matrix
        with torch.no_grad():
            predicted_adj_matrix = self.model(padded_features_tensor).squeeze(0).numpy()
        
        # Apply threshold to get binary adjacency matrix
        binary_adj_matrix = (predicted_adj_matrix > self.threshold).astype(int)
        
        # Return the binary adjacency matrix
        return binary_adj_matrix

# Example usage
# Assuming `model` and `max_nodes` are already defined and the model is trained

# Create an instance of the prediction model
prediction_model = FFNPredictionModel(model, max_nodes)

# Example parts set for debugging
example_parts = {Part(part_id=5, family_id=1), Part(part_id=7, family_id=1), Part(part_id=4, family_id=1)}

# Get the predicted binary adjacency matrix
predicted_binary_matrix = prediction_model.predict_adjacency_matrix(example_parts)

# Print the predicted binary adjacency matrix
print("Predicted Binary Adjacency Matrix:")
print(predicted_binary_matrix)

Predicted Binary Adjacency Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
